<a href="https://colab.research.google.com/github/tse-code/ML-AI-Projects/blob/main/DeepGAN_A_Deep_Learning_Framework_for_Generating_Realistic_Handwritten_Digits.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
# Import required libraries
# Import required libraries
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, LeakyReLU, BatchNormalization, Reshape, Flatten
from tensorflow.keras.models import Model, Sequential  # Ensure Sequential is imported from keras.models
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt


In [18]:
# Hyperparameters
NOISE_DIM = 100  # Dimension of noise vector
BATCH_SIZE = 64  # Batch size
EPOCHS = 5000  # Number of epochs
HALF_BATCH = BATCH_SIZE // 2  # Half batch for training discriminator


In [10]:
# Optimizer
adam = Adam(learning_rate=0.0002, beta_1=0.5)

In [1]:
# Generator Model
def build_generator():
    model = Sequential()
    model.add(Input(shape=(NOISE_DIM,)))  # Use Input() explicitly to avoid warnings
    model.add(Dense(256))
    model.add(LeakyReLU(0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(512))
    model.add(LeakyReLU(0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(1024))
    model.add(LeakyReLU(0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(28 * 28 * 1, activation='tanh'))
    model.add(Reshape((28, 28, 1)))
    return model

In [2]:
# Discriminator Model
def build_discriminator():
    model = Sequential()
    model.add(Input(shape=(28, 28, 1)))  # Use Input() explicitly
    model.add(Flatten())
    model.add(Dense(512))
    model.add(LeakyReLU(0.2))
    model.add(Dense(256))
    model.add(LeakyReLU(0.2))
    model.add(Dense(1, activation='sigmoid'))
    return model

In [11]:
# GAN Model
def build_gan(generator, discriminator):
    discriminator.trainable = False  # Freeze the discriminator for GAN training
    gan_input = Input(shape=(NOISE_DIM,))
    generated_img = generator(gan_input)
    gan_output = discriminator(generated_img)
    model = Model(gan_input, gan_output)
    model.compile(loss='binary_crossentropy', optimizer=adam)
    return model

# Build the generator
generator = build_generator()

# Build the GAN
gan = build_gan(generator, discriminator)

In [19]:
# Build the generator
generator = build_generator()



In [20]:
# Build the GAN
gan = build_gan(generator, discriminator)

In [21]:
# Load the dataset (for example, MNIST)
(X_train, _), (_, _) = tf.keras.datasets.mnist.load_data()
X_train = X_train / 127.5 - 1.0  # Normalize images between -1 and 1
X_train = np.expand_dims(X_train, axis=-1)

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


In [ ]:
# Training loop
d_losses = []
g_losses = []

for epoch in range(EPOCHS):
    epoch_d_loss = 0.0
    epoch_g_loss = 0.0

    for _ in range(X_train.shape[0] // BATCH_SIZE):
        # Step 1: Train Discriminator on real and fake data
        idx = np.random.randint(0, X_train.shape[0], HALF_BATCH)
        real_imgs = X_train[idx]
        noise = np.random.normal(0, 1, (HALF_BATCH, NOISE_DIM))
        fake_imgs = generator.predict(noise)

        real_y = np.ones((HALF_BATCH, 1)) * 0.9  # Label smoothing
        fake_y = np.zeros((HALF_BATCH, 1))

        # Train discriminator
        d_loss_real = discriminator.train_on_batch(real_imgs, real_y)
        d_loss_fake = discriminator.train_on_batch(fake_imgs, fake_y)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # Step 2: Train Generator
        noise = np.random.normal(0, 1, (BATCH_SIZE, NOISE_DIM))
        ground_truth_y = np.ones((BATCH_SIZE, 1))  # Generator tries to fool the discriminator

        discriminator.trainable = False  # Freeze discriminator for generator training
        g_loss = gan.train_on_batch(noise, ground_truth_y)

        # Ensure losses are scalar before adding
        epoch_d_loss += d_loss[0] if isinstance(d_loss, list) else d_loss
        epoch_g_loss += g_loss if isinstance(g_loss, (float, int)) else g_loss[0]

    # Average the losses for the epoch
    avg_d_loss = epoch_d_loss / (X_train.shape[0] // BATCH_SIZE)
    avg_g_loss = epoch_g_loss / (X_train.shape[0] // BATCH_SIZE)

    d_losses.append(avg_d_loss)
    g_losses.append(avg_g_loss)

    # Print losses for each epoch
    print(f"Epoch {epoch+1}/{EPOCHS}, Discriminator Loss: {avg_d_loss}, Generator Loss: {avg_g_loss}")

    # Save the generator model every 100 epochs
    if (epoch + 1) % 100 == 0:
        generator.save(f'generator_epoch_{epoch+1}.h5')

    # Display generated images every 500 epochs
    if (epoch + 1) % 500 == 0:
        display_images(generator)


    # Function to display generated images
def display_images(generator, examples=10, dim=(1, 10), figsize=(10, 1)):
    noise = np.random.normal(0, 1, (examples, NOISE_DIM))
    generated_imgs = generator.predict(noise)
    generated_imgs = 0.5 * generated_imgs + 0.5  # Rescale to [0, 1]

    plt.figure(figsize=figsize)
    for i in range(examples):
        plt.subplot(dim[0], dim[1], i + 1)
        plt.imshow(generated_imgs[i, :, :, 0], cmap='gray')
        plt.axis('off')
    plt.show()



    # Plot loss graph after training
def plot_loss(d_losses, g_losses):
    plt.figure(figsize=(10, 5))
    plt.plot(d_losses, label="Discriminator Loss")
    plt.plot(g_losses, label="Generator Loss")
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

# Call plot_loss function to display the loss graph
plot_loss(d_losses, g_losses)



